In [ ]:
!pip install pyopencl

## Kernel

```c
__kernel void calcMagnitude(__global float* vec, 
                            __global float* squares, 
                            __global float* magnitude)
{
    int id = get_global_id(0);
    int global_size = get_global_size(0);

    squares[id] = vec[id]*vec[id];
    
    barrier(CLK_GLOBAL_MEM_FENCE);

    if(id == 0) {
        *magnitude = 0;

        for(int i = 0; i < global_size; ++i) {
            *magnitude += squares[i];
        }
        *magnitude = sqrt(*magnitude);
    }
}
```

```c
__kernel void divide(__global float* vec,  __global float* magnitude) {

    int id = get_global_id(0);
    vec[id] = vec[id] / (*magnitude);
}
```

In [38]:
!touch program.cl

## Runtime

In [39]:
import numpy as np
import pyopencl as cl

np.random.seed(0)

In [ ]:
VEC_SIZE = 10000000
RANGE = 100

vec = np.random.uniform(-RANGE, RANGE, VEC_SIZE).astype(np.float32)
squares = np.zeros(VEC_SIZE).astype(np.float32)
magnitude = np.zeros(1, dtype=np.float32)

print(vec)
print(squares)
print(magnitude)

In [41]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)


In [43]:
mf = cl.mem_flags
vec_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=vec)
squares_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=squares)
magnitude_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=magnitude)


In [ ]:
program_file = open("program.cl", "r")
program_src = program_file.read()

program = cl.Program(ctx, program_src)
program.build()                              


In [45]:
kernel_magnitude = program.calcMagnitude
kernel_divide = program.divide

kernel_magnitude.set_args(vec_buf, squares_buf, magnitude_buf)
kernel_divide.set_args(vec_buf, magnitude_buf)


In [ ]:
global_size = (VEC_SIZE,)
local_size = (1000,)

cl.enqueue_nd_range_kernel(queue, kernel_magnitude, global_size, local_size)
cl.enqueue_nd_range_kernel(queue, kernel_divide, global_size, local_size)


In [ ]:
cl.enqueue_copy(queue, vec, vec_buf)

In [ ]:
print(vec)